# Определение ответственного лица для решения задачи

Данные предварительно должны быть размечены инструментом `label studio`

In [ ]:
!pip install tokenizers -q
!pip install transformers -q
!pip install transformers[torch] -q

## Анализ данных

In [34]:
import json
import re
import pandas as pd

from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, LineByLineTextDataset

Для корректной работы скрипта требуются файлы:
* tfs-titles.csv
* tfs-titles-mask.csv

### Заголовки артефактов

In [27]:
titles = pd.read_csv('../datasets/tfs-titles.csv')

titles.set_index('id', inplace=True)

titles.info()

titles.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1096 entries, 549475 to 627931
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   1096 non-null   object
dtypes: object(1)
memory usage: 17.1+ KB


,title
id,
549475,МРСК ЦиП и Центра. Анализ переполнения памяти ...
774271,[МОЭСК МС] #4990 Необходимо доработать логику ...
793494,[МОЭСК МС] #5184 Необходимо реализовать возмож...
709059,[МОЭСК]#HD1038238 Не отображаются ПКЗ в КСП.
781957,[МРСК ЦиП] #1764279 Необходима выгрузка всех с...


In [31]:
# уберём числовые значения
def normal_title(txt):
    return re.sub(r'\d+', '', txt)
    
titles.title = titles.title.apply(normal_title)

#### Создание tokenizer

In [39]:
# теперь создаём текстовый файл - corpus.txt
CORPUS_TXT='../datasets/corpus.txt'

# преобразуем в текстовый файл
with open(CORPUS_TXT, 'a', encoding='utf-8') as f:
    def read_rows_from_csv(row):
        f.write(f'{row["title"]}\n')

    titles.apply(read_rows_from_csv, axis=1)

    f.close()

In [43]:
tokenizer = BertWordPieceTokenizer()
tokenizer.train(files=CORPUS_TXT)

# убедится, что в каталоге есть папка tokenizer

tokenizer.save_model('../datasets/tokenizer')
tokenizer

Tokenizer(vocabulary_size=2339, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=None, lowercase=True, wordpieces_prefix=##)

### Обработка маскированного (размеченного) файла

In [48]:
titles_mask = pd.read_csv('../datasets/tfs-titles-mask.csv')

titles_mask.drop(['annotator', 'annotation_id', 'created_at', 'lead_time', 'updated_at'], axis=1, inplace=True)

titles_mask.set_index('id', inplace=True)

titles_mask.info()

titles_mask.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1096 entries, 549475 to 627931
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   1095 non-null   object
 1   title   1096 non-null   object
dtypes: object(2)
memory usage: 25.7+ KB


,label,title
id,,
549475,"[{""start"":0,""end"":17,""text"":""МРСК ЦиП и Центра...",МРСК ЦиП и Центра. Анализ переполнения памяти ...
774271,"[{""start"":17,""end"":218,""text"":""Необходимо дора...",[МОЭСК МС] #4990 Необходимо доработать логику ...
793494,"[{""start"":17,""end"":177,""text"":""Необходимо реал...",[МОЭСК МС] #5184 Необходимо реализовать возмож...
709059,"[{""start"":18,""end"":44,""text"":""Не отображаются ...",[МОЭСК]#HD1038238 Не отображаются ПКЗ в КСП.
781957,"[{""start"":20,""end"":53,""text"":""Необходима выгру...",[МРСК ЦиП] #1764279 Необходима выгрузка всех с...


Нужно размеченный тег сопоставить с текстом (title), используя 

In [14]:
def gen_train_data(file_path, save_path):
    """
    file_path: csv file exported by Label Studio
    save_path: save path
    """
    data = pd.read_csv(file_path)
    for idx, item in data.iterrows():
        text = item['title']
        if pd.isna(text):
            text = ''
        text_list = list(text)
        label_list = []
        labels = item['label']
        label_list = ['O' for i in range(len(text_list))]
        if pd.isna(labels):
            pass
        else:
            labels = json.loads(labels)
            for label_item in labels:
                start = label_item['start']
                end = label_item['end']
                label = label_item['labels'][0]
                label_list[start] = f'B-{label}'
                label_list[start+1:end-1] = [f'M-{label}' for i in range(end-start-2)]
                label_list[end - 1] = f'E-{label}'
        assert len(label_list) == len(text_list)
        with open(save_path, 'a', encoding='utf-8') as f:
            for idx_, line in enumerate(text_list):
                if text_list[idx_] == '\t' or text_list[idx_] == ' ':
                    text_list[idx_] = '，'
                line = text_list[idx_] + ' ' + label_list[idx_] + '\n'
                f.write(line)
            f.write('\n')

In [15]:
gen_train_data('../datasets/feature_titles-mark.csv', '../datasets/feature_titles-mark-finish.csv')

In [24]:
df = pd.read_csv('../datasets/feature_titles-mark.csv')

df.head()

,annotation_id,annotator,created_at,id,label,lead_time,title,updated_at
0,7,1,2023-08-25T04:25:34.057216Z,549475,"[{""start"":0,""end"":17,""text"":""МРСК ЦиП и Центра...",120.371,МРСК ЦиП и Центра. Анализ переполнения памяти ...,2023-08-25T04:33:53.491122Z
1,8,1,2023-08-25T04:25:55.656577Z,774271,"[{""start"":17,""end"":218,""text"":""Необходимо дора...",73.469,[МОЭСК МС] #4990 Необходимо доработать логику ...,2023-08-25T04:34:01.648770Z
2,9,1,2023-08-25T04:26:20.314071Z,793494,"[{""start"":17,""end"":177,""text"":""Необходимо реал...",58.721,[МОЭСК МС] #5184 Необходимо реализовать возмож...,2023-08-25T04:34:09.367064Z
3,10,1,2023-08-25T04:26:37.026182Z,709059,"[{""start"":18,""end"":44,""text"":""Не отображаются ...",57.769,[МОЭСК]#HD1038238 Не отображаются ПКЗ в КСП.,2023-08-25T04:34:18.382464Z
4,11,1,2023-08-25T04:26:51.503539Z,781957,"[{""start"":20,""end"":53,""text"":""Необходима выгру...",53.867,[МРСК ЦиП] #1764279 Необходима выгрузка всех с...,2023-08-25T04:34:26.530897Z


In [16]:
from datasets import load_dataset

raw_datasets = load_dataset("conll2003")

In [17]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [18]:
raw_datasets["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [21]:
pos_feature = raw_datasets["train"].features["pos_tags"]
pos_feature

Sequence(feature=ClassLabel(names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], id=None), length=-1, id=None)

In [22]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [20]:
chunk_feature = raw_datasets["train"].features["chunk_tags"]
chunk_feature

Sequence(feature=ClassLabel(names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], id=None), length=-1, id=None)